##  缺失数据补全

In [1]:
import numpy as np

from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score

In [2]:
rng = np.random.RandomState(0)

dataset = load_boston()
X_full,y_full = dataset.data,dataset.target
n_samples = X_full.shape[0]
n_features = X_full.shape[0]

# 交叉验证,预测
estimator = RandomForestRegressor(random_state=0,n_estimators=100)
score = cross_val_score(estimator,X_full,y_full).mean()
print "Score with entire dataset: %.2f"  % score

Score with entire dataset: 0.56


In [23]:
X_full.shape  # (506,13)
missing_rate = 0.75
n_missing_samples = np.floor(n_samples * missing_rate) # 379
missing_samples = np.hstack((np.zeros(n_samples - n_missing_samples,
                                     dtype=np.bool),
                            np.ones(n_missing_samples,
                                   dtype=np.bool)))
rng.shuffle(missing_samples)  #  打乱 missing_samples
missing_features = rng.randint(0,n_features,n_missing_samples) # 0-n_feature 之间, n_missing_samples 个缺失值



#  删除缺失值的剩余数据预测
X_filtered = X_full[~missing_samples,:]  # X_filtered.shape (127,3)
y_filtered = y_full[~missing_samples]
estimator = RandomForestRegressor(random_state=0,n_estimators=100)
score = cross_val_score(estimator,X_filtered,y_filtered).mean()
print "Score without the samples containing missing values = %.2f" % score

/home/hadoop/.local/lib/python2.7/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/hadoop/.local/lib/python2.7/site-packages/ipykernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Score without the samples containing missing values = 0.41


In [35]:
#  补全缺失值重新估计
X_missing = X_full.copy()
print np.where(missing_samples)[0]
print missing_features
X_missing[np.where(missing_samples)[0], missing_features] = 0
y_missing = y_full.copy()
estimator = Pipeline(['imputer',Imputer(missing_values=0,
                                       strategy='mean',
                                       axis=0),
                     'forest',RandomForestRegressor(random_state=0,
                                                   n_estimators=100)])
score = cross_val_score(estimator,X_missing,y_missing).mean()
print 'Score after imutation of the missing vales: %.2f' % score  

[  0   1   2   3   4   6   7  10  11  12  13  14  15  17  19  20  21  22
  24  26  27  29  30  31  32  33  34  35  37  38  39  40  42  43  44  45
  46  47  48  49  50  51  52  55  57  59  60  62  63  64  67  69  70  71
  73  74  75  76  77  78  80  81  82  83  84  85  86  87  88  89  92  94
  95  97  98  99 101 102 103 104 107 108 109 110 111 112 113 114 117 119
 120 122 123 124 125 126 127 128 129 132 134 136 137 140 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 163 164 165
 167 168 169 170 171 172 173 174 175 176 177 180 182 183 184 186 187 188
 189 190 191 192 193 194 195 198 199 202 203 204 205 206 207 211 212 214
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 233 234
 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 252 253 254
 256 257 263 265 267 268 269 270 271 272 276 279 280 281 282 284 288 290
 291 293 294 295 296 298 300 301 302 303 304 306 307 308 309 311 312 313
 314 315 316 317 319 321 323 324 325 326 327 328 33

IndexError: index 479 is out of bounds for axis 1 with size 13